# Scrape SATP data

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

# Set up
months = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]
years = list(range(2001, 2023 + 1))
letters_to_delete = ["“", "”", "‘", "’", "'", "\nRead less..."]
pattern = "|".join(map(re.escape, letters_to_delete))

def download_data(month, year):
    # Generate a dataset using the string
    theurl = f"https://www.satp.org/terrorist-activity/india-{month}-{year}"
    response = requests.get(theurl)
    soup = BeautifulSoup(response.content, 'html.parser')
    table = soup.find('table')  # Assuming the table of interest is the first one
    rows = table.find_all('tr')[1:]  # Skip the header row
    
    data = []
    for row in rows:
        date_cell = row.find('td', style="width: 15%;")
        if not date_cell:
            continue
        date = date_cell.get_text(strip=True)
        
        description_cell = row.find('td', colspan="2", style="width: 85%; font-family: Arial,Helvetica,sans-serif; font-size: small; text-align: justify; color: #294A6F;")
        if not description_cell:
            continue
        description_div = description_cell.find('div', class_="more")
        description = description_div.get_text(strip=True) if description_div else ""
        
        data.append([date, description, year])
    
    df = pd.DataFrame(data, columns=["date", "description", "year"])
    
    df["date"] = df["date"].str.replace("&nbsp", "", regex=False)
    df["date"] = df["date"].str.replace(" - ", " ", regex=False)
    df["description"] = df["description"].str.replace(pattern, "", regex=True)
    print(month,year)
    return df

# Download data from year 2000
year = 2000
datasets = [download_data(month, year) for month in months]
dataset_satp = pd.concat(datasets, ignore_index=True)
dataset_satp.columns = ["date", "description", "year"]

satp_new = dataset_satp.copy()

# Download data for all other years and append to 2000 data
for year in years:
    datasets = [download_data(month, year) for month in months]
    dataset_year = pd.concat(datasets, ignore_index=True)
    satp_new = pd.concat([satp_new, dataset_year], ignore_index=True)

satp_new["description"] = satp_new["description"].str.replace("Read less...", "", regex=False)
satp_new.to_csv("/Users/jonathanold/Library/CloudStorage/GoogleDrive-jonathan_old@berkeley.edu/My Drive/_Berkeley Research/Reservations and Conflict/Data/_gen/satp_scraped_python.csv", encoding='utf-8')


Jan 2000
Feb 2000
Mar 2000
Apr 2000
May 2000
Jun 2000
Jul 2000
Aug 2000
Sep 2000
Oct 2000
Nov 2000
Dec 2000
Jan 2001
Feb 2001
Mar 2001
Apr 2001
May 2001
Jun 2001
Jul 2001
Aug 2001
Sep 2001
Oct 2001
Nov 2001
Dec 2001
Jan 2002
Feb 2002
Mar 2002
Apr 2002
May 2002
Jun 2002
Jul 2002
Aug 2002
Sep 2002
Oct 2002
Nov 2002
Dec 2002
Jan 2003
Feb 2003
Mar 2003
Apr 2003
May 2003
Jun 2003
Jul 2003
Aug 2003
Sep 2003
Oct 2003
Nov 2003
Dec 2003
Jan 2004
Feb 2004
Mar 2004
Apr 2004
May 2004
Jun 2004
Jul 2004
Aug 2004
Sep 2004
Oct 2004
Nov 2004
Dec 2004
Jan 2005
Feb 2005
Mar 2005
Apr 2005
May 2005
Jun 2005
Jul 2005
Aug 2005
Sep 2005
Oct 2005
Nov 2005
Dec 2005
Jan 2006
Feb 2006
Mar 2006
Apr 2006
May 2006
Jun 2006
Jul 2006
Aug 2006
Sep 2006
Oct 2006
Nov 2006
Dec 2006
Jan 2007
Feb 2007
Mar 2007
Apr 2007
May 2007
Jun 2007
Jul 2007
Aug 2007
Sep 2007
Oct 2007
Nov 2007
Dec 2007
Jan 2008
Feb 2008
Mar 2008
Apr 2008
May 2008
Jun 2008
Jul 2008
Aug 2008
Sep 2008
Oct 2008
Nov 2008
Dec 2008
Jan 2009
Feb 2009
Mar 2009
A

In [ ]:
satp_new.reset_index(inplace=True)
satp_new['id_satp_new'] = 'id_'+ satp_new['index'].astype(str)

satp_new.to_csv("/Users/jonathanold/Library/CloudStorage/GoogleDrive-jonathan_old@berkeley.edu/My Drive/_Berkeley Research/Reservations and Conflict/Data/_gen/satp_scraped_python.csv", encoding='utf-8',index=False)


In [6]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

satp_new = pd.read_csv("/Users/jonathanold/Library/CloudStorage/GoogleDrive-jonathan_old@berkeley.edu/My Drive/_Berkeley Research/Reservations and Conflict/Data/_gen/satp_scraped_python.csv")
satp_new_orig = satp_new
satp_new = satp_new.iloc[:118399]

satp_new = satp_new.drop_duplicates(subset=['description','year','date'])
satp_new = satp_new.dropna(subset=['description'])
satp_new['d_new'] = satp_new['description'].str.strip()
satp_new = satp_new[['d_new','id_satp_new']]
satp_old = pd.read_csv("/Users/jonathanold/Library/CloudStorage/GoogleDrive-jonathan_old@berkeley.edu/My Drive/_Berkeley Research/Reservations and Conflict/Data/_gen/coding_events/data_sample_new.csv")
satp_old_orig = satp_old
satp_old = satp_old.drop_duplicates(subset=['description','year'])
satp_old = satp_old.dropna(subset=['description'])
satp_old['d_old'] = satp_old['description'].str.strip()
satp_old['id_satp_old'] = satp_old['id_satp'] 
satp_old = satp_old[['d_old','id_satp_old']]

# Define a function to transform the description
def transform_description(desc):
    desc = desc.lower()  # Convert to lower case
    desc = re.sub(r'[^a-z0-9\s]', '', desc)  # Remove special characters and punctuation
    return desc

# Apply the transformation to the description column
satp_old['dm_old'] = satp_old['d_old'].apply(transform_description)
satp_new['dm_new'] = satp_new['d_new'].apply(transform_description)


print("Read in data")
satp_old = satp_old[::-1]
print("Starting merge")



##### Do the actual merge
# 1. Merge based on d_new and d_old
merged_1 = pd.merge(satp_old, satp_new, left_on='d_old', right_on='d_new', how='outer', indicator=True, suffixes=('_old', '_new'))

# 2. Look at observations from satp_old that were NOT merged, and observations from satp_new that were NOT merged
unmerged_old_1 = merged_1[merged_1['_merge'] == 'left_only']
unmerged_new_1 = merged_1[merged_1['_merge'] == 'right_only']

merged_1b = merged_1[merged_1['_merge'] == 'both']
merged_1b['merge1'] = 1

# Create new DataFrames for second merge
unmerged_old_1 = unmerged_old_1[['id_satp_old', 'd_old', 'dm_old']]
unmerged_new_1 = unmerged_new_1[['id_satp_new', 'd_new', 'dm_new']]

# 3. Merge these based on dm_new and dm_old
merged_2 = pd.merge(unmerged_old_1, unmerged_new_1, left_on='dm_old', right_on='dm_new', how='outer', indicator=True, suffixes=('_old', '_new'))
merged_2b = merged_2[merged_2['_merge'] == 'both']
merged_2b['merge2'] = 1
# 4. Append to df_merge_final, which contains both merges
df_merge_final = pd.concat([merged_1b, merged_2b], ignore_index=True)

# Create DataFrames of unmerged observations from the second merge
unmerged_old_2 = merged_2[merged_2['_merge'] == 'left_only']
unmerged_new_2 = merged_2[merged_2['_merge'] == 'right_only']

# Drop the '_merge' column
df_merge_final.drop(columns=['_merge'], inplace=True)
unmerged_old_2 = unmerged_old_2.drop(columns=['_merge','d_new','dm_new'])
unmerged_new_2 = unmerged_new_2.drop(columns=['_merge','d_old','dm_old'])




from rapidfuzz import process, fuzz


# Function to perform fuzzy matching
def fuzzy_merge(df_1, df_2, key1, key2, threshold=90):
    """
    df_1 is the left table to join
    df_2 is the right table to join
    key1 is the key column of the left table
    key2 is the key column of the right table
    threshold is how close the matches should be to return a match, based on Levenshtein distance
    """
    matches = []
    scores = []
    # Iterate over the entries in df_1[key1]
    for value in df_1[key1]:
        # Skip matching if the value is null
        if pd.isnull(value):
            matches.append(None)
            scores.append(None)
        else:
            match, score, i = process.extractOne(value, df_2[key2], scorer=fuzz.token_sort_ratio)
            if match is not None:
                matches.append(match)
                scores.append(score)
            else:
                matches.append(None)
                scores.append(None)

    
    df_1['best_match'] = matches
    df_1['score'] = scores
    
    # Filter matches based on the threshold
    df_1_filtered = df_1[df_1['score'] >= threshold]
    
    # Perform the actual merge
    df_2.set_index(key2, inplace=True)
    merged_df = df_1_filtered.merge(df_2, left_on='best_match', right_index=True, how='outer', indicator=True)
    unmerged_old_3 = merged_df[merged_df['_merge'] == 'left_only']
    unmerged_new_3 = merged_df[merged_df['_merge'] == 'right_only']
    merged_df_filtered = merged_df[merged_df['_merge'] == 'both']
    return merged_df, unmerged_old_3, unmerged_new_3, merged_df_filtered

# Apply the fuzzy_merge function
fuzzy_merged_df, unmerged_old_3, unmerged_new_3, merged_df_filtered = fuzzy_merge(unmerged_old_2, unmerged_new_2, 'dm_old', 'dm_new', threshold=90)
merged_df_filtered['merge3'] = 1


df_merge_final = pd.concat([df_merge_final, merged_df_filtered], ignore_index=True)
df_merge_final['id_satp_old'] = df_merge_final['id_satp_old'].fillna(df_merge_final['id_satp_old_x'])
df_merge_final['id_satp_new'] = df_merge_final['id_satp_new'].fillna(df_merge_final['id_satp_new_y'])






print(df_merge_final['id_satp_old'].nunique())
print(df_merge_final['id_satp_new'].nunique())
df_merge_final['is_duplicate_new'] = df_merge_final.duplicated(subset='id_satp_new', keep=False)
df_merge_final['is_duplicate_old'] = df_merge_final.duplicated(subset='id_satp_old', keep=False)

# Filter: Drop duplicates (either side)
df_merge_new = df_merge_final.drop_duplicates(subset='id_satp_new', keep='first')
df_merge_old = df_merge_final.drop_duplicates(subset='id_satp_old', keep='first')

df_merge_new = df_merge_new[['id_satp_new','id_satp_old']]
df_merge_old = df_merge_old[['id_satp_new','id_satp_old']]

# Make final "new" dataset 
df_new = pd.merge(satp_new_orig, df_merge_new, on="id_satp_new", how="left", indicator=True)
df_orig_only = df_new[df_new['_merge'] == 'left_only']
df_new_only = df_new[df_new['_merge'] == 'right_only']
df_both_only = df_new[df_new['_merge'] == 'both']

df_new = df_new[['date','year','description','id_satp_old','id_satp_new']]
# Make final "old" dataset 
df_old = pd.merge(satp_old_orig, df_merge_old, left_on="id_satp", right_on="id_satp_old", how="outer", indicator=True)
df_orig_only = df_old[df_old['_merge'] == 'left_only']
df_new_only = df_old[df_old['_merge'] == 'right_only']
df_both_only = df_old[df_old['_merge'] == 'both']


# Merge Final dataset with training data
df_training = pd.read_csv("/Users/jonathanold/Library/CloudStorage/GoogleDrive-jonathan_old@berkeley.edu/My Drive/_Berkeley Research/Reservations and Conflict/Data/_gen/coding_events/sample_events_coded_new.csv")
df_training = df_training.iloc[:, 3:]

variables_list = ['relevant_event', 'multiple_events','internal_conflict']  # replace with your actual variable names
# Keep only the variables which are non-missing in any of the variables in variables_list
df_training= df_training.dropna(subset=variables_list, how='all')

df_training_final = pd.merge(df_new, df_training, left_on='id_satp_old', right_on='id_satp',how='left')


df_training_final.to_csv("/Users/jonathanold/Library/CloudStorage/GoogleDrive-jonathan_old@berkeley.edu/My Drive/_Berkeley Research/Reservations and Conflict/Data/_gen/coding_events/satp_training_data.csv")


Read in data
Starting merge


/var/folders/yz/pz74ttxj0g9ggdqz9nnmm5q00000gn/T/ipykernel_3484/1330238046.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_1b['merge1'] = 1
/var/folders/yz/pz74ttxj0g9ggdqz9nnmm5q00000gn/T/ipykernel_3484/1330238046.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_2b['merge2'] = 1
/var/folders/yz/pz74ttxj0g9ggdqz9nnmm5q00000gn/T/ipykernel_3484/1330238046.py:119: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_in

118267
118322


NameError: name 'df_merge_final' is not defined

In [ ]:
# !pip install rapidfuzz 
filtered_df = df_merge_final[(df_merge_final['is_duplicate_old'] == True) | (df_merge_final['is_duplicate_new'] == True)]


In [ ]:
print("Starting")
inspect_merge = pd.merge(satp_new, final_merged, left_on='id_satp_new', right_on='id_satp_new', how='left', indicator=True)
# Filter out rows that are only in df2 (right_only)
df2_not_in_df1 = inspect_merge[inspect_merge['_merge'] == 'right_only']


inspect_merge2 = pd.merge(satp_old, final_merged, left_on='id_satp', right_on='id_satp',  how='left', indicator=True)
# Filter out rows that are only in df2 (right_only)
old_not_in_df1 = inspect_merge2[inspect_merge2['_merge'] == 'left_only']


Starting


In [ ]:
duplicates_df1 = satp_new[satp_new.duplicated(subset=['description'], keep=False)]


In [ ]:
unmerged_new_2.columns

Index(['id_satp_old', 'one_old', 'id_satp_new', 'one_new', 'd_new'], dtype='object')